In [1]:
import cv2
import os
from os import listdir
from os.path import isfile, join
import numpy as np

import cv2
import time
import os

import traceback

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import load_model

import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
def captureImages(name,path):
    try:
        cam = cv2.VideoCapture(0)
        capture = False

        cv2.namedWindow("Capturing Images ...")

        img_counter = 0
        timeout = time.time() + 60*1   # 5 minutes from now
        while True:
            test = 0
            if test == 3 or time.time() > timeout:
                break
            test = test - 1
            ret, frame = cam.read()
            cv2.imshow("Capturing Images from different angles", frame)
            if not ret:
                break
            k = cv2.waitKey(1)
            if img_counter==20:  # Change the number of Images to capture 
                break;
            if k%256 == 27:
                # ESC pressed
                print("Escape hit, closing...")
                break
            time.sleep(1)
            capture = True
            if capture:
                img_name = "{}.jpg".format(img_counter)
                cv2.imwrite(os.path.join(path , img_name),frame)
                img_counter += 1
                capture = False

        cam.release()

        cv2.destroyAllWindows()
        return True
    except Exception as ex:
        track = traceback.format_exc()
        print(track)
        return False

In [3]:
def registerUser(UserName):
    path = registerDirectoryPath + UserName
    
    try:
        os.makedirs(path)
        if captureImages(UserName,path):
            print(UserName + ' is registered')
            return True
        else:
            print(UserName + ' is not successfully registered')
            return False
    except OSError:
        print ("Creation of the register directory %s failed" % path)  

In [4]:
def detectFacesFromRegisteredImages(gray,frame):
    global region_of_interest_frame
    global face_cascade
    region_of_interest_frame = None
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray,1.05,6)   #Returns 4 arguments x,y coordinate of top left corner and width and height
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x-50,y-50),(x+w+50,y+h+50),(255,0,0),2)
        region_of_interest_gray = gray[y:y+h,x:x+w]   # Cascade will be applied in gray scaled version of region of  interest
        region_of_interest_frame = frame[y-50:y+h+50,x-50:x+w+50] # Rectangle will be drawn to colored version of area of interset
        
    return frame,region_of_interest_frame

In [5]:
def processImages(userName):
    sourcePath = registerDirectoryPath + userName
    #print(sourcePath)
    path = trainingDirectoryPath + userName
    #print(path)
    try:
        image_files = [file for file in listdir(sourcePath) if isfile(join(sourcePath,file))]
        os.makedirs(path)
        for i,image in enumerate(image_files):
            actualImage = cv2.imread(sourcePath + '\\' + image)
            grayImage = cv2.cvtColor(actualImage,cv2.COLOR_BGR2GRAY) #Converting the picture GRAY here as detection happens using gray scale
            canvas,picture = detectFacesFromRegisteredImages(grayImage,actualImage)
            imageName = "{}.jpg".format(userName + str(i))
            try:
                processedImage = cv2.cvtColor(picture,cv2.COLOR_BGR2GRAY)
                success = cv2.imwrite(os.path.join(path , imageName),processedImage)
                if not success:
                    os.remove(os.path.join(path , imageName))
            except:
                continue
    except Exception as e:
        print('Image Processing failed!')
        track = traceback.format_exc()
        print(track)

In [6]:
trainingDirectoryPath = os.getcwd() + "\\Training_Images\\"

def startLearning():
    
    print('Gathering training data ...')
    
    y=[]
    trainingFolderInfo = {}
    for folder in list(os.walk(trainingDirectoryPath))[0][1]:
        userFolder = trainingDirectoryPath + '\\' + folder + '\\'
        numberOfImages = len([imageFile for imageFile in os.listdir(userFolder) if os.path.isfile(os.path.join(userFolder, imageFile))])
        trainingFolderInfo[folder] = numberOfImages
    for i,folder in enumerate(trainingFolderInfo.keys()):
        labelList = list(np.repeat(i, trainingFolderInfo[folder]))
        y.extend(labelList)
    y = np.array(y)
    
    
    
    X = []
    IMG_HEIGHT, IMG_WIDTH = 64, 64
    for folder in list(os.walk(trainingDirectoryPath))[0][1]:
        userFolder = trainingDirectoryPath + '\\' + folder + '\\'
        image_files = [ imageFile for imageFile in listdir(userFolder) if isfile(join(userFolder,imageFile))]
        x_empty = np.empty((len(image_files), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
        #x_empty = np.empty((IMG_HEIGHT, IMG_WIDTH,len(image_files)), dtype='float32')
        for i,nameOfImage in enumerate(image_files):
            # Read the image one by one
            image = userFolder + nameOfImage
            actualImage = cv2.imread(image,flags=cv2.IMREAD_UNCHANGED).astype("int16").astype('float32')
            resizedImage = cv2.resize(actualImage, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
            # Normalizing the values of the images 
            normalizedImage = (resizedImage - np.min(resizedImage)) / (np.max(resizedImage) - np.min(resizedImage))
            # store the final values in x_empty which is right now empty 
            x_empty[i] = normalizedImage
            X.append(x_empty[i])
    X = np.array(X)
    
    print('Training data collected!')
    
    # Reshaping X and y
    y_categorical = to_categorical(y)
    X_reshaped = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
    
    print('Building neural network...')
    global classifier
    classifier = Sequential()

    # Step 1 - adding Convolution layer of input 64x64x3 (as it is RGB) with 32 filters each with size 3x3
    classifier.add(Conv2D(64, (3, 3), input_shape = (64, 64,1), activation = 'relu'))

    # Step 2 - adding the max Pooling layer
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    # Adding a second convolutional layer with 32 filters each with size 3x3
    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    # Step 3 - Flattening
    classifier.add(Flatten())

    # Step 4 - Full connection
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 64, activation = 'relu'))
    
    classifier.add(Dense(units = len(list(os.walk(trainingDirectoryPath))[0][1]), activation = 'softmax'))
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    print('Neural Network created!')
    
    print('Neural Network starts learning...')
    classifier.fit(x=X_reshaped,y=y_categorical,epochs=10,batch_size=2) 
    print('Learning finished!')
    return classifier,dict(list(zip(list(np.unique(y)),list(os.walk(trainingDirectoryPath))[0][1])))

In [7]:
def recognizeFaceOnVideo(gray,frame):
    faces = face_cascade.detectMultiScale(gray,1.05,6)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x-50,y-50),(x+w+50,y+h+50),(255,0,0),2)
        region_of_interest_gray = gray[y:y+h,x:x+w]   # Cascade will be applied in gray scaled version of region of  interest
        region_of_interest_frame = frame[y-50:y+h+50,x-50:x+w+50] # Rectangle will be drawn to colored version of area of interest
        
        try:
            picture_64x64 = cv2.resize(region_of_interest_frame,(64,64),interpolation = cv2.INTER_AREA) #Resize the image to 64x64
            picture_64x64_gray = cv2.cvtColor(picture_64x64,cv2.COLOR_BGR2GRAY) #Convert to gray
            picture_64x64_gray_normalized =  (picture_64x64_gray - np.min(picture_64x64_gray)) / (np.max(picture_64x64_gray) - np.min(picture_64x64_gray)) #Normalize
      
            test_picture = np.array(picture_64x64_gray_normalized)
            test_picture = np.expand_dims(test_picture,axis=0)
            test_picture = test_picture.reshape(test_picture.shape[0],test_picture.shape[1],test_picture.shape[2],1)
        
            result = classifier.predict(test_picture)
            
            
            recognizedFaceName = user_label[np.argmax(result[0])]
            cv2.putText(region_of_interest_frame, recognizedFaceName, (x-200, y-40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            cv2.putText(region_of_interest_frame, 'Accuracy: ' + str(round(result[0][np.argmax(result[0])]*100,2)) + '%', (x-200, y-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (0,255,0), 2)
            
        except:
            continue
    return frame

In [8]:
def openCameraToRecognizeFace():
    
    video_capture = cv2.VideoCapture(0)
    timeout = time.time() + 60*1   # 1 minutes from now
    while video_capture.isOpened():
        while True:
            try:
                test = 0
                if test == 3 or time.time() > timeout:
                    print('Disconnecting the camera!')
                    break
                test = test - 1
                _,frame = video_capture.read()
                #print(frame)
                gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                canvas = recognizeFaceOnVideo(gray,frame)
                cv2.imshow('Video',canvas)
                if cv2.waitKey(1) & 0xFF==ord('q'):
                    print('Disconnecting the camera!')
                    break
            except Exception as e:
                print('Camera Failed to recognise your face!')
                
                track = traceback.format_exc()
                print(track)
                continue
        video_capture.release()
        cv2.destroyAllWindows()

In [9]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end="\r")
        time.sleep(1)
        t -= 1
    print('Starting Camera!!!')
    return True

In [10]:
def createDir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass
    except OSError:
        print ("Creation of the directory %s failed" % path)

In [11]:
def start():
    name = input('Your name: ')
    global registerDirectoryPath
    global trainingDirectoryPath
    global user_label
    registerDirectoryPath = os.getcwd() + "\\Registered_Images\\"
    trainingDirectoryPath = os.getcwd() + "\\Training_Images\\"
    createDir(registerDirectoryPath)
    createDir(trainingDirectoryPath)
    createDir(trainingDirectoryPath + '\\Dummy\\')
    dummyImage = cv2.imread('dummyUser.jpg')
    for i in range(0,5):
        dummyGrayImage = cv2.cvtColor(dummyImage, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(os.path.join(trainingDirectoryPath + '\\Dummy\\', 'dummyUser{}.jpg'.format(i)),dummyGrayImage)
    if os.path.exists(registerDirectoryPath + name):
        print('You are already registered')
        classifier = load_model('NN_FaceRecognitionData.h5')
        with open(os.getcwd() + '\\Label_User_information.pickle', 'rb') as handle:
            label_user_dict = pickle.load(handle)
        user_label = label_user_dict
        print('Loading Trained Neural Network Model...')
        classifier = load_model('NN_FaceRecognitionData.h5')
        print('Neural Network loaded!')
        
        print('Camera will open after sometime to recognise you!')
        openCameraToRecognizeFace()
        
    else:
        print('As you are not registered. Please follow as directed.')
        print('Camera will open up to register your face!\nRequest you to face the webcam')
        startCountdown = int(input('Enter the timer (in seconds) here to start capturing your image: '))
        if countdown(startCountdown):
            print('Capturing...')
            registerUser(name)
        print('Processing Your Images for training...')
        processImages(name)
        print('Image Processing finished!')
        print('Algorithm is learning your Image...')
        classifier,label_user_dict = startLearning()
        
        if os.path.exists(os.getcwd() + "\\NN_FaceRecognitionData.h5"):
            os.remove(os.getcwd() + "\\NN_FaceRecognitionData.h5")
        
        
        if os.path.exists(os.getcwd() + "\\Label_User_information.pickle"):
            os.remove(os.getcwd() + "\\Label_User_information.pickle")
        
        
        classifier.save("NN_FaceRecognitionData.h5")
        with open(os.getcwd() + '\\Label_User_information.pickle', 'wb') as handle:
            pickle.dump(label_user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        user_label = label_user_dict
        print('Algorithm learnt your Image!')
        
        print('Loading Trained Neural Network Model...')
        classifier = load_model('NN_FaceRecognitionData.h5')
        print('Neural Network loaded!')
        print('Camera will open after sometime to recognise you!')
        
        openCameraToRecognizeFace()

In [14]:
start()

Your name: Aakash
As you are not registered. Please follow as directed.
Camera will open up to register your face!
Request you to face the webcam
Enter the timer (in seconds) here to start capturing your image: 5
Starting Camera!!!
Capturing...
Aakash is registered
Processing Your Images for training...
Image Processing finished!
Algorithm is learning your Image...
Gathering training data ...
Training data collected!
Building neural network...
Neural Network created!
Neural Network starts learning...
Epoch 1/10
45/45 [==============================] - 1s 24ms/step - loss: 1.2369 - accuracy: 0.4889
Epoch 2/10
45/45 [==============================] - 1s 22ms/step - loss: 0.7768 - accuracy: 0.7111
Epoch 3/10
45/45 [==============================] - 1s 22ms/step - loss: 0.1998 - accuracy: 0.9333
Epoch 4/10
45/45 [==============================] - 1s 21ms/step - loss: 0.0134 - accuracy: 1.0000
Epoch 5/10
45/45 [==============================] - 1s 21ms/step - loss: 0.0011 - accuracy: 1.0000